In [93]:
import numpy as np
import cv2 as cv
import random
import os

class InputPic:
    before = None
    after = None
    dir_path = '/home/pi/AI/API/UUIDDirectory'
    
    def setimg(self, inputimg):
        os.chdir('/home/pi/AI/API')
        self.before = cv.imread(inputimg, 4)
        self.after = cv.resize(self.before,(1000,300))
        pass 
    
    def getimg(self):
        return self.after
    
    def saveimg(self, name, uuid):
        os.chdir('/home/pi/AI/API/UUIDDirectory/'+ uuid +"/after/")
        cv.imwrite(str(name)+ "_resize.png", self.after)
        pass

    def beforesize(self):
        print('변경 전 사이즈는 ', self.before.shape)
        pass
    
    def aftersize(self):
        print('변경 후 사이즈는 ', self.after.shape)
        pass
    
    def createFolder(directory):
        os.chdir(dir_path)
        os.makedirs(directory)
        os.chdir(dir_path+'/'+directory)
        os.makedirs('before')
        os.makedirs('after')
        pass
    
    def deleteFolder(path):
        os.rmdir(dir_path +'/'+ path + '/before')

if __name__ == "__main__":
    pic=[]
    uuid = 'abcdefghijk'
    InputPic.createFolder(uuid)

    for i in range(0,3):
        pic.append(InputPic())
        pic[i].setimg(str(i)+'.png')
        
    for i in range(0,3):
        pic[i].saveimg(i,uuid)
        
    for i in range(0,3):
        pic[i].beforesize()
        
    for i in range(0,3):
        pic[i].aftersize()
        
    InputPic.deleteFolder(uuid)
    pass

변경 전 사이즈는  (648, 1152, 3)
변경 전 사이즈는  (648, 1152, 3)
변경 전 사이즈는  (648, 1152, 3)
변경 후 사이즈는  (300, 1000, 3)
변경 후 사이즈는  (300, 1000, 3)
변경 후 사이즈는  (300, 1000, 3)
